Input a shapefile of the polygon you would like to clip, input the raster file you are clipping from, set the output directory and output file name, then run all cells. The code will tell you if the polygon doesn't overlap the raster, otherwise it will clip the raster to only the area of the polygon.

In [46]:
import os, sys, rasterio, fiona

#polygon_file = 'polygon.shp'
#raster_file = 'raster.tif'
#output_directory = 'output_folder'
#output_file = 'output_raster.tif'

polygon_file = 'D:\Palmer2020 Data Repo\ROI shapefiles\Dream.shp'
raster_file = 'D:\Palmer2020 Data Repo\Imagery datasets\DigitalGlobe products\QuickBird2 [2004-2013]\QB02_20041215135202_1010010003742B00_04DEC15135202-P1BS-052281612010_01_P001_u08rf3031.tif'
output_directory = 'C:\i_am_a_demo_delete_me'
output_file = output_directory + '/{p}-{t}'.format(p=polygon_file.split('\\')[-1], t=raster_file.split('\\')[-1])

try:
    os.makedirs(output_directory)
    print("output directory: " + output_directory)
except: pass


output directory: C:\i_am_a_demo_delete_me


In [47]:
from rasterio.mask import mask

with fiona.open(polygon_file,'r')as shp:
    shape=[feature["geometry"] for feature in shp]

output_file = output_directory + '/{p}-{t}'.format(p=polygon_file.split('\\')[-1], t=raster_file.split('\\')[-1])

error_list = []
with rasterio.open(raster_file) as tile:
    try:
        out_image, out_transform = rasterio.mask.mask(tile, shape, crop=True)
    except ValueError:
        print(f"No overlap found for {tile.files[0]}. Skipping.")
    except:
        error_list.append([raster_file, sys.exc_info()[0]])
        print("unknown error: " + sys.exc_info()[0])
    else:
        out_meta = tile.meta
        out_meta.update({"driver": "GTiff",
              "height": out_image.shape[1],
              "width": out_image.shape[2],
              "transform": out_transform})

        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)
print(error_list)

[]
